In [1]:
import os
import pandas as pd
import sqlite3
#import matplotlib.pyplot as plt
from classify_hems import check_vfr
from classify_hems import check_ifr

import classify

base_name = 'FH50'
sect_names = ['FH50.base', 'FH50_1.1', 'FH50_1.2', 'FH50_1.3', 'FH50_1.4', 'FH50_1.5', 'FH50_1.6', 'FH50_1.7', 'FH50_1.8']
dbname = "db/hems-fh50.sqlite"

#sect_names = ['FH10.base', 'FH10.1', 'FH10.2', 'FH10.3', 'FH10.6', 'FH10.7', 'FH10.8']
#dbname = "hems-fh10.sqlite"

In [2]:
sect_df = pd.read_csv('data/FH-base_sektorit.csv', sep=',')
sect_df.set_index('Sector_no', inplace=True, drop=False)
geo_df = pd.read_csv('data/Saahavaintoasemat.csv', sep=',', index_col='ICAO')

In [3]:
con = sqlite3.connect(dbname)

In [4]:
sect_df[['Sector_no', 'dep_dest', 'sect_wx', 'cld_ceiling', 'alt_open', 'alt_all']].to_sql('sect', con, if_exists='replace', index=False)

In [5]:
vfr_allowing_classes = ['DAY HEMS VFR', 'SUNDOWN HEMS VFR', 'NIGHT HEMS VFR']
ifr_allowing_classes = ['DAY HEMS IFR', 'DAY HEMS VFR RETURN NIGHT IFR', 'NIGHT HEMS IFR']
icing_classes = ['DAY IFR ICING', 'NIGHT IFR ICING']

In [6]:
output_path=f'output/{base_name}.csv'
if os.path.exists(output_path):
    open(output_path, 'w').close()  # clear file

for sect_name in sect_names:
    print(f"reading sector {sect_name}")  # (< 1 min)
    df = classify.read_sector_df(con, sect_name)
    if df.empty:
        print(f"❌ WARNING: No data for sector {sect_name}. Skipping.")
        continue

    icao_term = sect_df.loc[sect_name, 'dep_dest']
    alt_open = sect_df.loc[sect_name, 'alt_open'].split(';')
    alt_all = sect_df.loc[sect_name, 'alt_all'].split(';')
    cld_ceiling = sect_df.loc[sect_name, 'cld_ceiling']

    #print(f"reading alts for {sect_name}")  # (< 10 s)
    #alts = alt_open
    #alt_df = classify.read_alt_df(con, alts)
    df.dropna(inplace=True)  # TODO don't drop all na?

    print(f"classifying VFR for {sect_name}")  # (< 10 s)
    df['vfr_class'] = df.apply(check_vfr, axis=1)
    #print(f"classifying IFR for {sect_name}")  # (> 6 mins)
    #df['ifr_class'] = df.apply(lambda row: check_ifr(row, alt_df, alts, cld_ceiling), axis=1)

    print("creating month an hour columns")
    df['month'] = df.time.str.slice(5, 7).astype(int)
    df['hour'] = df.time.str.slice(11, 13).astype(int)

    print("creating VFR column")
    df['VFR'] = df.vfr_class.isin(vfr_allowing_classes).astype(int)
    #print("creating IFR column")
    #df['IFR'] = df.ifr_class.isin(ifr_allowing_classes).astype(int) & (df.VFR == 0)
    #print("creating ICING column")
    #df['NO FLY ICING'] = df.ifr_class.isin(icing_classes).astype(int) & (df.VFR == 0) & (df.IFR == 0)
    #print("creating NO FLY column")
    #df['NO FLY'] = (df.VFR == 0) & (df.IFR == 0) & (df['NO FLY ICING'] == 0)

    print(f"building hourly summary for {sect_name}")
    #hourly = classify.make_hourly_csv(df, sect_name)
    hourly = classify.make_hourly_csv_vfr_only(df, sect_name)

    hourly.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    #hourly.to_csv(f'output/canfly-{sect_name}.csv')

con.close()

reading sector FH50.base
classifying VFR for FH50.base
creating month an hour columns
creating VFR column
building hourly summary for FH50.base
reading sector FH50_1.1
classifying VFR for FH50_1.1
creating month an hour columns
creating VFR column
building hourly summary for FH50_1.1
reading sector FH50_1.2
classifying VFR for FH50_1.2
creating month an hour columns
creating VFR column
building hourly summary for FH50_1.2
reading sector FH50_1.3
classifying VFR for FH50_1.3
creating month an hour columns
creating VFR column
building hourly summary for FH50_1.3
reading sector FH50_1.4
classifying VFR for FH50_1.4
creating month an hour columns
creating VFR column
building hourly summary for FH50_1.4
reading sector FH50_1.5
classifying VFR for FH50_1.5
creating month an hour columns
creating VFR column
building hourly summary for FH50_1.5
reading sector FH50_1.6
classifying VFR for FH50_1.6
creating month an hour columns
creating VFR column
building hourly summary for FH50_1.6
reading se